In [2]:
# Importing required libraries 
import pandas as pd
import os 
import csv

In [3]:
# Creating main directory to store all the files 
main_dir_path = 'C:/Users/TejYadav/Desktop/All_Files'
os.mkdir(main_dir_path)

In [3]:
# Function to get all the paths for a particular file name. Checks all the directories and sub-directories 

def get_paths(folder_name,file_name):
    
    path = "C:/Users/Tej Yadav/Desktop/V codes/Data Files for Machine Learning/"+(folder_name)+"/"

    all_paths = []
    
    for r, d, f in os.walk(path):
        for file in f:
            if (file_name) in file:
                all_paths.append(os.path.join(r, file))
   
    return all_paths

In [4]:
def new_columns(fname,cnames):
    new_cnames = []
    for i in cnames:
        new_cnames.append(fname+"_"+i)
    return new_cnames

In [5]:
# To generate the output file for a particular file name present in a folder named folder name and extract attributes defined by 
# attribute name 

# output file format - main_dir_path + sheet_dir_path + input_file_name.csv
def generate_file(folder_name,file_names,attribute_name,input_name):
    
    dframe_file = pd.DataFrame()
    output_dframe = pd.DataFrame()
    
    attribute_name = attribute_name.encode('utf-8')
    attribute_name = attribute_name.strip()
    attribute_names = attribute_name.split("\n")
    
        
    for fname in file_names:
        all_paths_files = get_paths(folder_name,fname)
        
        if len(file_names)>1:
            temp_fname = fname.replace(".csv","")
            new_attribute_names = new_columns(temp_fname,attribute_names)

        for j in range(len(all_paths_files)):
            dframe = pd.read_csv(all_paths_files[j])
            
            if j == 0:
                    dframe_file = dframe.loc[:,attribute_names]

                    if len(file_names)>1:
                        dframe_file.columns = new_attribute_names
            else:
                temp_df = dframe.loc[:,attribute_names] 
                if len(file_names)>1:
                    temp_df.columns = new_attribute_names
                dframe_file.append(temp_df,ignore_index=True)

        output_dframe = pd.concat([output_dframe,dframe_file],axis=1)
        
    return output_dframe    

In [6]:
# To read input xlsx file 

xls = pd.ExcelFile('C:\Users\TejYadav\Desktop\V codes\WIP_V2.1.xlsx')

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-6-a912433b5c0c>, line 3)

In [7]:
# Getting names of all the worksheets from a workbook 

all_sheet_names = xls.sheet_names
print(all_sheet_names)

NameError: name 'xls' is not defined

In [8]:
# Function that uses other functions to extract particular attributes 

def extract_cols(sheets):
    global main_dir_path
    for sheet_name in sheets:
        print(sheet_name)
        
        if str(sheet_name) != "System Pump":
            
            dframe_sheet = pd.read_excel(xls,sheetname=sheet_name)
            records,cols = (dframe_sheet.shape)
            
            dframe_for_each_sheet = pd.DataFrame()
            result_df = pd.DataFrame()
            
            for i in range(0,records):
                
                folder_name = str(dframe_sheet.loc[i,'Folder Name'])
                file_name = str(dframe_sheet.loc[i,'File Name'])
                file_names = file_name.split("\n")
                attribute_name = (dframe_sheet.loc[i,'Column Name'])
                input_name = str(dframe_sheet.loc[i,'Inputs'])
                if isinstance(attribute_name, unicode):
                
                    result_df = generate_file(folder_name,file_names,attribute_name,input_name)
                
                    dframe_for_each_sheet = pd.concat([dframe_for_each_sheet,result_df],axis=1)
                    
            final_records,final_cols=(dframe_for_each_sheet.shape)
            output_file_path = str(main_dir_path + '/' + sheet_name + '_all_attributes.csv')
            o_file = open(output_file_path,mode="w")
            dframe_for_each_sheet.to_csv(output_file_path,sep=',', encoding='utf-8',index = False)
            print(output_file_path)
            print(final_cols)

        elif str(sheet_name) == "System Pump":
            
            dframe_sheet = pd.read_excel(xls,sheetname=sheet_name)
            dframe_for_each_sheet = pd.DataFrame()
            result_df = pd.DataFrame()
            
            records,cols = (dframe_sheet.shape)
            
            for i in range(0,9):
               
                folder_name = str(dframe_sheet.loc[i,'Folder Name'])
                file_name = str(dframe_sheet.loc[i,'File Name'])
                file_names = file_name.split("\n")
                attribute_name = (dframe_sheet.loc[i,'Column Name'])
                
                if isinstance(attribute_name, unicode) and folder_name != ' ' and folder_name != 'nan' and file_name != ' ' and file_name != 'nan':
                    result_df = generate_file(folder_name,file_names,attribute_name,input_name)
                    dframe_for_each_sheet = pd.concat([dframe_for_each_sheet,result_df],axis=1)
                   
            # To consider these 15 files for each of the inputs 
            
            system_pump_file_names = ['crah_server1_A_1.csv', 'crah_server1_A_2.csv', 'crah_server1_A_3.csv', 'crah_server1_A_4.csv',
                                    'crah_server1_B_1.csv', 'crah_server1_B_2.csv', 'crah_server1_B_3.csv', 'crah_server1_B_4.csv', 'crah_storage1_A_1.csv', 'crah_storage1_B_1.csv',
                                    'crah_ups_A_1.csv', 'crah_ups_A_2', 'crah_ups_B_1.csv', 'crah_ups_B_2.csv', 'crah_wan1_A_1.csv']
            
            for i in range(9,records):
               
                folder_name = str(dframe_sheet.loc[i,'Folder Name'])
                file_name = str(dframe_sheet.loc[i,'File Name'])
                file_names = file_name.split("\n")
                attribute_name = (dframe_sheet.loc[i,'Column Name'])
                input_name = str(dframe_sheet.loc[i,'Inputs'])
                
                if file_name != 'nan' and folder_name != 'nan' and file_name != ' ' and folder_name != ' ' and isinstance(attribute_name,unicode) :
                    data_available = True
                    result_df = generate_file(folder_name,system_pump_file_names,attribute_name,input_name)
                    dframe_for_each_sheet = pd.concat([dframe_for_each_sheet,result_df],axis=1)
                
            final_records,final_cols=(dframe_for_each_sheet.shape)
            
            output_file_path = str(main_dir_path + '/' + sheet_name + '_all_attributes.csv')
            o_file = open(output_file_path,mode="w")
            dframe_for_each_sheet.to_csv(output_file_path,sep=',', encoding='utf-8',index = False)
            print(output_file_path)
            print(final_cols)

In [15]:
# Calling function for considering all the sheets in the input workbook
print("Output files generated")
extract_cols(all_sheet_names)

Output files generated
Condenser Pump
C:/Users/TejYadav/Desktop/All_Files/Condenser Pump_all_attributes.csv
0
Evaporator Pump
C:/Users/TejYadav/Desktop/All_Files/Evaporator Pump_all_attributes.csv
0
System Pump
C:/Users/TejYadav/Desktop/All_Files/System Pump_all_attributes.csv
0
Cooling Tower
C:/Users/TejYadav/Desktop/All_Files/Cooling Tower_all_attributes.csv
0
Compression Chiller
C:/Users/TejYadav/Desktop/All_Files/Compression Chiller_all_attributes.csv
0
